In [1]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
print(tf.__version__)

import keras
print(keras.__version__)

import numpy as np
print(np.__version__)

2.15.0
2.15.0
1.25.2


In [2]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d navoneel/brain-mri-images-for-brain-tumor-detection
import zipfile
zip_file_path = '/content/brain-mri-images-for-brain-tumor-detection.zip'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/brain_tumor_dataset')

Dataset URL: https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection
License(s): copyright-authors
 60% 9.00M/15.1M [00:00<00:00, 92.8MB/s]
100% 15.1M/15.1M [00:00<00:00, 132MB/s] 


In [4]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121, VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate, Dense, GlobalAveragePooling2D, BatchNormalization, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


import tensorflow as tf
import keras.backend as K
import os
import random
import csv
import shutil
from keras.models import save_model

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
from keras.models import load_model
from keras.layers import Dropout
from keras.models import Model


In [6]:
import os
import random
import csv

# Path to the data folder containing the two class folders (yes and no)
data_folder = "brain_tumor_dataset"

# List to store image paths and corresponding labels
data = []

# Loop through each class folder
for class_label in ["yes", "no"]:
    class_folder = os.path.join(data_folder, class_label)
    # Get list of image files in the folder
    images = os.listdir(class_folder)
    # Iterate through each image
    for img in images:
        # Append image path and corresponding label to the data list
        data.append((os.path.join(class_label, img), 1 if class_label == "yes" else 0))

# Shuffle the data
random.shuffle(data)

# Path to save the CSV file
csv_file = "data.csv"

# Write data to CSV file
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    # Write header
    writer.writerow(['Image', 'Label'])
    # Write data rows
    writer.writerows(data)

print("CSV file created successfully.")


CSV file created successfully.


In [7]:
# Function to load data from a CSV file
def load_data_from_csv(csv_file):
    data = []
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        header = next(reader)  # Skip header
        for row in reader:
            data.append(row)
    return data

# Function to shuffle and split data into train, validation, and test sets
def split_data(data, test_ratio=0.3, validation_ratio=0.2):
    random.shuffle(data)
    test_size = int(test_ratio * len(data))
    validation_size = int(validation_ratio * len(data))
    test_data = data[:test_size]
    remaining_data = data[test_size:]
    validation_data = remaining_data[:validation_size]
    train_data = remaining_data[validation_size:]
    return train_data, validation_data, test_data

# Function to copy images to destination folder
def copy_images(data, destination_folder, source_folder="brain_tumor_dataset"):
    for row in data:
        image_path = row[0]
        label = row[1]
        class_folder = os.path.join(destination_folder, "yes" if label == '1' else "no")
        os.makedirs(class_folder, exist_ok=True)
        shutil.copy(os.path.join(source_folder, image_path), class_folder)

# Load data from CSV file
csv_file = "data.csv"
data = load_data_from_csv(csv_file)

# Split data into train, validation, and test sets
train_data, validation_data, test_data = split_data(data)

# Create folders for train, validation, and test data
train_folder = "Training"
validation_folder = "Validation"
test_folder = "Testing"
os.makedirs(train_folder, exist_ok=True)
os.makedirs(validation_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Copy train, validation, and test images to respective folders
copy_images(train_data, train_folder)
copy_images(validation_data, validation_folder)
copy_images(test_data, test_folder)

print("Train, validation, and test data copied to respective folders.")


Train, validation, and test data copied to respective folders.


In [8]:
# Constants
img_width, img_height = 224, 224
batch_size = 64
input_shape = (img_width, img_height, 3)
num_classes = 2  # For binary classification

# Set up data generators
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
validation_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Set up train, validation, and test generators
train_generator = train_datagen.flow_from_directory(
    'Training',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    'Validation',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    'Testing',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False  # Don't shuffle test data for evaluation
)

# Print class indices for reference
print("Class Indices:")
print(train_generator.class_indices)

# Fetch batches of data directly from generators
X_train, y_train = next(train_generator)
X_val, y_val = next(validation_generator)
X_test, y_test = next(test_generator)

# Resize the images
X_train = tf.image.resize(X_train, (28, 28))
X_val = tf.image.resize(X_val, (28, 28))
X_test = tf.image.resize(X_test, (28, 28))


Found 128 images belonging to 2 classes.
Found 50 images belonging to 2 classes.
Found 75 images belonging to 2 classes.
Class Indices:
{'no': 0, 'yes': 1}


In [9]:
def overfit(history):
  # Get the final training and validation losses
  final_train_loss = history.history['loss'][-1]
  final_val_loss = history.history['val_loss'][-1]

  # Print the final losses
  print(f'Final Training Loss: {final_train_loss}')
  print(f'Final Validation Loss: {final_val_loss}')

  # Check for overfitting
  if final_val_loss > final_train_loss:
      print('The model is overfitting.')
  else:
      print('The model is not overfitting.')

In [48]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Input

class MonteCarloDropoutImproved:
    def __init__(self):
        pass

    def build_model(self, model_file_path, dropout_rate=0.5):
        model = self.__load_model(model_file_path)
        input_layer = model.input
        x = input_layer
        index = 0

        for layer in model.layers:
            if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.Dense):
                x = Dropout(dropout_rate, name=f"dropout_{index}")(x, training=True)
                index += 1
            x = layer(x)

        self.model = Model(input_layer, x)

    def predict_monte_carlo_dropout(self, input_samples, sampling_num):
        predictions = []

        for _ in range(sampling_num):
            pred = self.model.predict(input_samples)
            predictions.append(pred)

        predictions = np.array(predictions)
        pred_avg = np.average(predictions, axis=0)
        pred_std = np.std(predictions, axis=0)

        return pred_avg, pred_std

    def frequency_distribution(self, input_sample, sampling_num):
        input_samples = np.tile(input_sample, (sampling_num, 1, 1, 1))
        predictions = self.model.predict(input_samples)

        return predictions

    def __load_model(self, model_file_path):
        model = tf.keras.models.load_model(model_file_path)
        return model

In [61]:
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout

tfd = tfp.distributions

class VariationalInferenceNN:
    def __init__(self):
        self.variational_model = None

    def build_variational_model(self, base_model):
        inputs = base_model.input
        outputs = base_model.output
        num_samples = tf.placeholder_with_default(input=1, shape=())

        def posterior_mean_field(kernel_size, bias_size=0, dtype=None):
            n = kernel_size + bias_size
            c = tf.math.log(tf.expm1(1.))
            return tf.keras.Sequential([
                tfp.layers.VariableLayer(2 * n, dtype=dtype),
                tfp.layers.DistributionLambda(lambda t: tfd.Independent(
                    tfd.Normal(loc=t[..., :n],
                               scale=1e-5 + tf.nn.softplus(c + t[..., n:])),
                    reinterpreted_batch_ndims=1)),
            ])

        prior = tfd.Independent(tfd.Normal(loc=tf.zeros_like(tf.reshape(outputs, (-1,))),
                                           scale=1.0), reinterpreted_batch_ndims=1)

        posterior = posterior_mean_field(kernel_size=outputs.shape[-1], dtype=tf.float32)

        y = tfp.layers.DenseVariational(outputs.shape[-1], posterior, prior, kl_weight=num_samples / tf.cast(50, tf.float32))(inputs)
        variational_model = Model(inputs=[inputs, num_samples], outputs=y)
        self.variational_model = variational_model

    def compile_model(self, optimizer='adam', loss='mse'):
        if self.variational_model is None:
            raise ValueError("Variational model is not built yet. Call build_variational_model() first.")
        self.variational_model.compile(optimizer=optimizer, loss=loss)

    def fit(self, x_train, y_train, num_samples=10, epochs=10, batch_size=32, validation_data=None):
        if self.variational_model is None:
            raise ValueError("Variational model is not built yet. Call build_variational_model() first.")
        self.variational_model.fit([x_train, num_samples], y_train, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    def evaluate(self, x_test, y_test):
        if self.variational_model is None:
            raise ValueError("Variational model is not built yet. Call build_variational_model() first.")
        return self.variational_model.evaluate([x_test, 1], y_test)

    @staticmethod
    def load_model(model_file_path):
        return tf.keras.models.load_model(model_file_path)


**Simple CNN**

In [55]:
def create_simple_cnn(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),  # Adding dropout regularization
        Dense(num_classes, activation='softmax')
    ])
    return model

# Create the modified model
model = create_simple_cnn(input_shape, num_classes)

# Compile the model with reduced learning rate and early stopping
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Define early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# Train the model with the early stopping callback
start_time = time.time()
history = model.fit(train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=10,  # Increased epochs for better convergence
                    validation_data=validation_generator,
                    validation_steps=len(validation_generator),
                    callbacks=[early_stopping])  # Using early stopping
end_time = time.time()

Epoch 1/10
3/4 [=====================>........] - ETA: 0s - loss: 0.6668 - accuracy: 0.5729

4/4 [==============================] - 3s 201ms/step - loss: 0.7710 - accuracy: 0.5781 - val_loss: 0.5231 - val_accuracy: 0.7600
Epoch 2/10
4/4 [==============================] - 1s 140ms/step - loss: 0.6313 - accuracy: 0.7188 - val_loss: 0.5208 - val_accuracy: 0.7733
Epoch 3/10
4/4 [==============================] - 1s 151ms/step - loss: 0.5416 - accuracy: 0.7656 - val_loss: 0.4734 - val_accuracy: 0.7867
Epoch 4/10
4/4 [==============================] - 1s 150ms/step - loss: 0.4550 - accuracy: 0.8047 - val_loss: 0.4607 - val_accuracy: 0.7467
Epoch 5/10
4/4 [==============================] - 1s 150ms/step - loss: 0.4039 - accuracy: 0.8281 - val_loss: 0.4411 - val_accuracy: 0.8133
Epoch 6/10
4/4 [==============================] - 1s 147ms/step - loss: 0.3715 - accuracy: 0.8672 - val_loss: 0.4385 - val_accuracy: 0.8133
Epoch 7/10
4/4 [==============================] - 1s 141ms/step - loss: 0.3371 - accuracy: 0.8750 - val_loss: 0.4270 - val_accuracy: 0.8133
Epoch 8/10
4/4 [===============

In [56]:
training_time = end_time - start_time

# Print training time
print(f"Training time: {training_time} seconds")

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator, verbose=0)
print(f'Test accuracy: {test_acc}')

# Print Training and Validation Accuracy
train_acc = history.history['accuracy'][-1]
val_acc = history.history['val_accuracy'][-1]
print(f'Training Accuracy: {train_acc}')
print(f'Validation Accuracy: {val_acc}')

# Check for overfitting
overfit(history)


Training time: 8.206925868988037 seconds
Test accuracy: 0.8266666531562805
Training Accuracy: 0.9296875
Validation Accuracy: 0.8266666531562805
Final Training Loss: 0.23398305475711823
Final Validation Loss: 0.42405518889427185
The model is overfitting.


In [49]:

# Save the model
model_path = '/content/models/simple_cnn_model.h5'
save_model(model, model_path)
# Create an instance of MonteCarloDropout and load the saved model
mcd_model = MonteCarloDropoutImproved()
mcd_model.build_model(model_path)

# Load your original model for comparison and get the input shape
original_model = keras.models.load_model(model_path)
original_input_shape = original_model.input_shape

# Resize the test images to match the input shape of the original model
resized_X_test = tf.image.resize(X_test, (224, 224))

original_loss, original_accuracy = original_model.evaluate(resized_X_test, y_test)

# Set the number of Monte Carlo Dropout samples
sampling_num = 100

# Get true labels (assuming from a generator like test_generator)
true_labels = test_generator[0][1]

pred_avg, pred_std = mcd_model.predict_monte_carlo_dropout(resized_X_test, sampling_num)

pred_labels = np.argmax(pred_avg, axis=1)
new_accuracy = np.mean(pred_labels == true_labels)


2/2 [==============================] - 0s 17ms/step


In [50]:
print('new_accuracy',new_accuracy)

new_accuracy 0.640625


In [21]:
from tensorflow import keras

# Constants
img_width, img_height = 224, 224
batch_size = 32
input_shape = (img_width, img_height, 3)
epochs = 10

# Set up data generators
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    '/content/Training',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    '/content/Testing',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 128 images belonging to 2 classes.
Found 75 images belonging to 2 classes.


In [58]:
import time
import numpy as np
import tensorflow as tf

# Resize test_generator.labels to match resized_X_test
if resized_X_test.shape[0] != len(test_generator.labels):
    new_labels = np.array([label for label, _ in zip(test_generator.labels, range(resized_X_test.shape[0]))])
    true_labels = new_labels  # Update true_labels with the resized labels
else:
    true_labels = test_generator.labels  # Use the original labels

# Resize validation_generator.labels to match X_val
if X_val.shape[0] != len(validation_generator.labels):
    new_val_labels = np.array(validation_generator.labels[:X_val.shape[0]])
else:
    new_val_labels = validation_generator.labels

# Measure time taken for Bayesian method
start_time_bayesian = time.time()

# Set the number of Monte Carlo Dropout samples
sampling_num = 100

pred_avg, pred_std = mcd_model.predict_monte_carlo_dropout(resized_X_test, sampling_num)
resized_X_val = tf.image.resize(X_val, (224, 224))  # Adjust the size as needed

# Calculate new accuracy
pred_labels = np.argmax(pred_avg, axis=1)
new_accuracy = np.mean(pred_labels == true_labels)

# Calculate validation accuracy and test accuracy for Bayesian method
val_pred_labels = np.argmax(mcd_model.predict_monte_carlo_dropout(resized_X_val, sampling_num), axis=1)
# Ensure new_val_labels has compatible shape for comparison
val_accuracy_bayesian = np.mean(val_pred_labels == new_val_labels[:len(val_pred_labels)])  # Use slicing to match shapes

test_accuracy_bayesian = np.mean(pred_labels == true_labels)

# Measure time taken for Bayesian method
end_time_bayesian = time.time()
bayesian_method_time = end_time_bayesian - start_time_bayesian

# Calculate composite score
time_weight = 0.4  # 40% weight for time
train_acc_weight = 0.1  # 10% weight for training accuracy
val_acc_weight = 0.3  # 30% weight for validation accuracy
test_acc_weight = 0.2  # 20% weight for test accuracy

composite_score = (
    (1 / bayesian_method_time) * time_weight +
    train_acc * train_acc_weight +
    val_accuracy_bayesian * val_acc_weight +
    test_accuracy_bayesian * test_acc_weight
)

# Print Bayesian method results and composite score
print(f"New Accuracy (Bayesian Method): {new_accuracy}")
print(f"Validation Accuracy (Bayesian Method): {val_accuracy_bayesian}")
print(f"Test Accuracy (Bayesian Method): {test_accuracy_bayesian}")
print(f"Time taken for Bayesian Method: {bayesian_method_time} seconds")
print(f"Composite Score (Bayesian Method): {composite_score}")


2/2 [==============================] - 0s 20ms/step
New Accuracy (Bayesian Method): 0.640625
Validation Accuracy (Bayesian Method): 0.0
Test Accuracy (Bayesian Method): 0.640625
Time taken for Bayesian Method: 21.77505397796631 seconds
Composite Score (Bayesian Method): 0.2394633944750379


In [52]:
resized_X_train = tf.image.resize(X_train, (224, 224))  # Adjust the size as needed

In [57]:
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
tf.keras.backend.clear_session()

# Assuming y_train and y_val are your label arrays
y_train_one_hot = to_categorical(y_train, num_classes=2)
y_val_one_hot = to_categorical(y_val, num_classes=2)

# Build and compile your model
mcd_model = MonteCarloDropoutImproved()
mcd_model.build_model(model_file_path='/content/models/simple_cnn_model.h5')  # Replace 'your_model_path.h5' with your model file
mcd_model.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Now you can evaluate your model using the resized input data and one-hot encoded labels
train_loss, train_accuracy = mcd_model.model.evaluate(resized_X_train, y_train_one_hot, verbose=0)
val_loss, val_accuracy = mcd_model.model.evaluate(resized_X_val, y_val_one_hot, verbose=0)

# Check for overfitting
if train_accuracy > val_accuracy:
    print("Warning: Potential overfitting detected.")
else:
    print("No overfitting detected.")

# Print training and validation accuracies
print("Training Accuracy:", train_accuracy)
print("Validation Accuracy:", val_accuracy)


Training Accuracy: 0.609375
Validation Accuracy: 0.5799999833106995


**Bayes by Backprop**

In [ ]:

# Define the model architecture
inputs = Input(shape=input_shape)
x = Conv2D(32, kernel_size=3, padding='same', activation='relu')(inputs)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)  # Add dropout after Conv2D layer
x = Conv2D(64, kernel_size=3, padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)  # Add dropout after Conv2D layer
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout after Dense layer
outputs = Dense(num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


from tensorflow.keras.callbacks import EarlyStopping
# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


# Train the model
history = model.fit(train_generator, epochs=10, validation_data=validation_generator)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')


# Get the final training and validation losses
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]

# Print the final losses
print(f'Final Training Loss: {final_train_loss}')
print(f'Final Validation Loss: {final_val_loss}')

# Check for overfitting
if final_val_loss > final_train_loss:
    print('The model is overfitting.')
else:
    print('The model is not overfitting.')

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_generator)

# Evaluate the model
train_acc = model.evaluate(train_generator)[1]
val_acc = model.evaluate(validation_generator)[1]
test_acc = model.evaluate(test_generator)[1]

print(f'Training Acc: {train_acc}')
print(f'Validation Acc: {val_acc}')
print(f'Test Acc: {test_acc}')

Found 128 images belonging to 2 classes.
Found 50 images belonging to 2 classes.
Found 75 images belonging to 2 classes.
Epoch 1/10
2/2 [==============================] - 7s 1s/step - loss: 1.5790 - accuracy: 0.5547 - val_loss: 0.4904 - val_accuracy: 0.7800
Epoch 2/10
2/2 [==============================] - 1s 615ms/step - loss: 0.9043 - accuracy: 0.5312 - val_loss: 0.7860 - val_accuracy: 0.3800
Epoch 3/10
2/2 [==============================] - 1s 608ms/step - loss: 0.8881 - accuracy: 0.5547 - val_loss: 0.4390 - val_accuracy: 0.7800
Epoch 4/10
2/2 [==============================] - 1s 614ms/step - loss: 0.7717 - accuracy: 0.6797 - val_loss: 0.4375 - val_accuracy: 0.7000
Epoch 5/10
2/2 [==============================] - 1s 410ms/step - loss: 0.7787 - accuracy: 0.6875 - val_loss: 0.4406 - val_accuracy: 0.8200
Epoch 6/10
2/2 [==============================] - 1s 419ms/step - loss: 0.5816 - accuracy: 0.7109 - val_loss: 0.5271 - val_accuracy: 0.7600
Epoch 7/10
2/2 [==========================

**2D cnn**

In [ ]:
# Define the model
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Record the start time
start_time = time.time()

# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)

# Record the end time
end_time = time.time()

# Calculate the training time
training_time = end_time - start_time
print(f"Training Time: {training_time} seconds")

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')

# Print process time and metrics
print(f'Time taken: {end_time - start_time} seconds')
print(f'Training Acc: {np.mean(history.history["accuracy"])}')
print(f'Validation Acc: {np.mean(history.history["val_accuracy"])}')
print(f'Test Acc: {test_acc}')




Epoch 1/10
2/2 [==============================] - 5s 2s/step - loss: 7.9992 - accuracy: 0.4219 - val_loss: 0.8350 - val_accuracy: 0.7000
Epoch 2/10
2/2 [==============================] - 1s 585ms/step - loss: 4.3070 - accuracy: 0.4844 - val_loss: 5.3258 - val_accuracy: 0.3000
Epoch 3/10
2/2 [==============================] - 1s 334ms/step - loss: 3.3262 - accuracy: 0.4297 - val_loss: 1.7409 - val_accuracy: 0.3000
Epoch 4/10
2/2 [==============================] - 1s 359ms/step - loss: 1.1170 - accuracy: 0.4297 - val_loss: 0.7795 - val_accuracy: 0.3000
Epoch 5/10
2/2 [==============================] - 1s 370ms/step - loss: 0.6696 - accuracy: 0.5469 - val_loss: 0.6237 - val_accuracy: 0.8600
Epoch 6/10
2/2 [==============================] - 1s 383ms/step - loss: 0.6343 - accuracy: 0.7578 - val_loss: 0.6120 - val_accuracy: 0.8400
Epoch 7/10
2/2 [==============================] - 1s 346ms/step - loss: 0.6217 - accuracy: 0.7891 - val_loss: 0.5802 - val_accuracy: 0.8600
Epoch 8/10
2/2 [=======

Final Training Loss: 0.47866290807724
Final Validation Loss: 0.38188162446022034
The model is not overfitting.


In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


**Monte Carlo Dropout**

In [ ]:
import os
import keras
from keras.layers import Dropout
from keras.models import Model
from keras import backend as K
import numpy as np
class MontecarloDropout:
    def __init__(self):
        return

    def build_model(self, model_file_path):
        """
        keras modelからMontecarloDropoutに対応したモデルを作成
        build monte carlo dropout model base on keras_model.
        """

        model = self.__load_model(model_file_path)

        # change dropout layer to dropout layer that can use dropout in inference.
        # ドロップアウト層を推論時にもドロップアウトできるドロップアウト層に変更する。
        for ily, layer in enumerate(model.layers):
            # input layer
            if ily == 0:
                input = layer.input
                h = input
            # is dropout layer ?
            if 'dropout' in layer.name:
                # change dropout layer
                h = Dropout(layer.rate)(h, training=True)
            else:
                h = layer(h)

        self.model = Model(input, h)

        return

    def md_predict(self, xs, sampling_num):
        """
        predict with using monte carlo dropout sampling.
        return prediction average, std

        xs : input sample array. xs = x0, x1, x2, ...
        """

        pre_ys = []
        for ismp in range(sampling_num):
            pre_y = self.model.predict(xs)
            pre_ys.append(pre_y)
        pre_ys = np.array(pre_ys)

        # calculate ave, std
        pre_ave = np.average(pre_ys, axis=0)
        pre_std = np.std(pre_ys, axis=0)

        return pre_ave, pre_std

    def md_freq_dist(self, x, sampling_num):
        """
        frequency distribution of prediction with using monte carlo dropout sampling.
        return predict(x), predict(x), ..., predict(x)

        x : one input sample

        """

        xs = np.ones((sampling_num, *x.shape)) * x
        pre_y = self.model.predict(xs)

        return pre_y

    def __load_model(self, model_file_path):
        """
        load model .h5 file
        """
        model = keras.models.load_model(model_file_path)
        return model

In [ ]:
# Define the model
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)

Epoch 1/10
2/2 [==============================] - 5s 1s/step - loss: 4.8896 - accuracy: 0.5312 - val_loss: 4.9059 - val_accuracy: 0.3000
Epoch 2/10
2/2 [==============================] - 1s 390ms/step - loss: 2.4503 - accuracy: 0.5781 - val_loss: 0.4002 - val_accuracy: 0.8600
Epoch 3/10
2/2 [==============================] - 1s 400ms/step - loss: 0.8714 - accuracy: 0.7812 - val_loss: 0.3188 - val_accuracy: 0.8400
Epoch 4/10
2/2 [==============================] - 1s 384ms/step - loss: 0.5648 - accuracy: 0.7266 - val_loss: 0.7021 - val_accuracy: 0.6200
Epoch 5/10
2/2 [==============================] - 1s 485ms/step - loss: 0.4790 - accuracy: 0.7266 - val_loss: 0.4067 - val_accuracy: 0.8000
Epoch 6/10
2/2 [==============================] - 1s 556ms/step - loss: 0.3091 - accuracy: 0.8906 - val_loss: 0.2872 - val_accuracy: 0.9200
Epoch 7/10
2/2 [==============================] - 1s 589ms/step - loss: 0.3072 - accuracy: 0.8906 - val_loss: 0.2671 - val_accuracy: 0.9200
Epoch 8/10
2/2 [=======

In [ ]:
model.save('2D_CNN.h5')


In [ ]:
mcd = MontecarloDropout()
mcd.build_model('2D_CNN.h5')

# Make predictions with Monte Carlo Dropout on test data
pre_ave, pre_std = mcd.md_predict(test_generator[0][0], sampling_num=10)

# Ground truth labels for the test data
true_labels = test_generator[0][1]

# Convert probabilities to binary predictions (0 or 1)
binary_predictions = (pre_ave > 0.5).astype(int)

# Calculate accuracy
test_accuracy = np.mean(binary_predictions == true_labels)

# Print accuracy
print(f"Test Acc: {test_accuracy * 100:.2f}%")


2/2 [==============================] - 0s 14ms/step
Test Acc: 56.45%
